In [33]:
IMAGE_SIZE = [224, 224]
TRAINING_PATH = '/home/dilara/UGI-endoscopy/training-data'
TESTING_PATH = '/home/dilara/UGI-endoscopy/testing-data/video3'
CLASSES = ['pylorus', 'retroflex-stomach', 'z-line', 'other2']
CLASSES_TEST = ['pylorus', 'retroflex-stomach', 'z-line', 'other']

import keras
from keras.utils import np_utils
from keras.layers import Dense, Flatten 
from keras.models import Model, Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
import glob
import matplotlib.pyplot as plt
import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import skimage
from skimage import io
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sbs
from PIL import Image

## setting the paths
all_paths = []
len_lis = []
for landmark in CLASSES:
    other_path = os.path.join(TRAINING_PATH, landmark, '*')
    all_paths.append(sorted(glob.glob(other_path)))
# all_paths = list(np.concatenate(all_paths).flat)

## finding the number of image per class
for lis in all_paths:
    x = 0
    for item in lis:
        x+=1
    print(x)
    len_lis.append(x)
print(len_lis)

all_paths = list(np.concatenate(all_paths).flat)
# df = pd.read_csv('file-names/filtered-names/numbered-labels.csv')
# all_labels = df['Finding'].to_numpy()

# # pylorus_labels = all_labels[0:5]
# # retroflex_stomach_labels = all_labels[999:1004]
# # zline_labels = all_labels[1763:1768]
# # other_labels = all_labels[2695:2700]

pylorus_labels = [1] * len_lis[0]
retroflex_stomach_labels = [3] * len_lis[1]
zline_labels = [2] * len_lis[2]
other_labels = [0] * len_lis[3]
all_labels = list(np.concatenate([pylorus_labels, retroflex_stomach_labels,
 zline_labels, other_labels]).flat)
all_labels = np.array(all_labels)

6993
5366
6524
6321
[6993, 5366, 6524, 6321]


In [16]:
## Augmenting the Dataset
# for path in all_paths:
# 	file_name = path[:len(path)-4]
# 	image = cv2.imread(path)
# 	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# 	rows, cols, dim = image.shape

# 	# flipping images upside down and leftright
# 	image_fliplr = np.fliplr(image)
# 	image_flipud = np.flipud(image)
# 	plt.imsave(f'{file_name}_image_flipud.jpg', image_flipud)
# 	plt.imsave(f'{file_name}_image_fliplr.jpg', image_fliplr)

# 	# rotating image
# 	rotated_img = skimage.transform.rotate(image, 90)
# 	plt.imsave(f'{file_name}_rotated_img.jpg', rotated_img)

# 	# shear transform on x-axis
# 	M = np.float32([[1, 0.7, 0],
# 					[0, 1  , 0],
# 					[0, 0  , 1]])             
# 	sheared_img = cv2.warpPerspective(image,M,(int(cols*1.5),int(rows*1.5)))
# 	plt.imsave(f'{file_name}_sheared_img.jpg', sheared_img)

# 	## zoom out
# 	M = np.float32([[1.5, 0  , 0],
# 					[0,   1.5, 0],
# 					[0,   0,   1]])
# 	scaledin_img = cv2.warpPerspective(image,M,(cols,rows))
# 	plt.imsave(f'{file_name}_scaledin_img.jpg', scaledin_img)

# 	M = np.float32([[0.8, 0  , 0],
# 					[0,   0.8, 0],
# 					[0,   0,   1]])
# 	scaledout_img = cv2.warpPerspective(image,M,(cols,rows))
# 	plt.imsave(f'{file_name}_scaledout_img.jpg', scaledout_img)

In [30]:
# Renaming the images
all_paths_test = []

for landmark in CLASSES_TEST:
    other_path = os.path.join(TESTING_PATH, landmark, '*')
    all_paths_test.append(sorted(glob.glob(other_path)))

all_paths_test = list(np.concatenate(all_paths_test).flat)

for path in all_paths_test:
    old = path
    new = f'{path[:len(path)-4]}_video3.jpg'
    os.rename(old, new)